In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import os

In [2]:
browser = webdriver.Chrome('/usr/local/bin/chromedriver') 

In [3]:
browser.get('https://s3.amazonaws.com/tripdata/index.html')

In [4]:
all_links = [link.get_attribute('href') for link in browser.find_elements_by_tag_name('a')]

In [5]:
linksJC_18 = [link for link in all_links if 'JC' in link if '2018' in link]
linksJC_17 = [link for link in all_links if 'JC' in link if '2017' in link]
links_18 = [link for link in all_links if 'JC' not in link if '2018' in link]
links_17 = [link for link in all_links if 'JC' not in link if '2017' in link]

In [6]:
def create_df(link):
    
    url = urllib.request.urlopen(link)
    output = open('temporary.zip', 'wb')    
    output.write(url.read())
    output.close()
    dataframe = pd.read_csv('temporary.zip')
    
    if (len(dataframe.columns) == 15):
        dataframe.columns = ['Trip Duration (sec)', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth_Year', 'Gender']
    else:
        dataframe.columns = ['Trip Duration (sec)', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID',
       'Localized Value', 'User Type', 'Birth_Year', 'Gender']
        dataframe = dataframe.drop('Localized Value',1)
        
    print(link)
    os.remove('temporary.zip')
    
    return dataframe

In [7]:
def prepare_df(df):
    
#     drop n/a
    df = df.dropna(how='any').reset_index(drop=True)
    
#     change data types
    df['Birth_Year'] = df.Birth_Year.astype(int)
    df['Start Time'] = pd.to_datetime(df['Start Time'])
    df['Stop Time'] = pd.to_datetime(df['Stop Time'])
    
#     add Dge column
    df['Age'] = 2020 - df['Birth_Year']
    
#     exclude ages > 90 years
    df = df[df['Age'] < 90]
    
    return df

In [8]:
ny18 = pd.DataFrame()
ny17 = pd.DataFrame()

In [9]:
for link in links_18:
    temporary_df = create_df(link)
    ny18 = ny18.append(temporary_df, ignore_index=True, sort=False)

https://s3.amazonaws.com/tripdata/201801-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201802-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201803-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201804-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201805-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201806-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201807-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201808-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201809-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201810-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201811-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201812-citibike-tripdata.csv.zip


In [10]:
for link in links_17:
    temporary_df = create_df(link)
    ny17 = ny17.append(temporary_df, ignore_index=True, sort=False)

https://s3.amazonaws.com/tripdata/201701-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201702-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201703-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201704-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201705-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201706-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201707-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201708-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201709-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201710-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201711-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201712-citibike-tripdata.csv.zip


In [11]:
ny17 = prepare_df(ny17)
ny18 = prepare_df(ny18)

In [12]:
nydf = ny17.append(ny18, ignore_index=True, sort=False)

In [15]:
nydf.to_csv('../tableau.csv', encoding='utf-8', index=False)